#### Using a different approach to deal with mid-year and year-end Type deprication. Also want to standardize/format registrant name and client name text.

In [1]:
!pip install category_encoders

    100% |████████████████████████████████| 92kB 5.5MB/s ta 0:00:011
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [44]:
!pip install eli5

    100% |████████████████████████████████| 102kB 3.5MB/s a 0:00:01
    100% |████████████████████████████████| 51kB 30.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for tabulate ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/2b/67/89/414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
Successfully built tabulate
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import re 
import datetime as dt
import eli5
import category_encoders as ce
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures             #, RobustScaler
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.model_selection import GridSearchCV, cross_val_score                #, cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from eli5.sklearn import PermutationImportance
from sklearn.pipeline import make_pipeline
import pickle
#from google.colab import drive

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

## Load and Prepare Data for Model

In [3]:
lob = pd.read_csv('All_Years.csv')
lob.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (24,34,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,ID,Year,Received,Amount,Type,Period,RegistrantID,RegistrantName,GeneralDescription,Address,RegistrantCountry,RegistrantPPBCountry,ClientName,GeneralDescription2,ClientID,SelfFiler,ContactFullname,IsStateOrLocalGov,ClientCountry,ClientPPBCountry,ClientState,ClientPPBState,Code,SpecificIssue,LobbyistName,LobbyistCoveredGovPositionIndicator,OfficialPosition,GovEntityName,ForeignEntityName,ForeignEntityCountry,ForeignEntityPPBcountry,ForeignEntityContribution,ForeignEntityOwnershipPercentage,ForeignEntityStatus,AffiliatedOrgName,AffiliatedOrgCountry,AffiliatedOrgPPBCcountry,ActivityInformation,AffiliatedOrgsURL,RegistrationEffectiveDate,TerminationEffectiveDate
0,0,83792D41-7B9E-40E4-87AA-B7828C79DCAC,1999,1/1/1999 0:00,NaN,REGISTRATION,UNDETERMINED,24624,MCLEOD WATKINSON & MILLER,Lobbying and government affairs,ONE MASSACHUSETTS AVE. NW\nSUITE 800\nWASHINGT...,USA,USA,CALIFORNIA AVOCADO COMMISSION,NaN,36,False,MICHAEL R. MCLEOD,False,USA,USA,CALIFORNIA,CALIFORNIA,AGRICULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,A9623E72-F519-4927-AFD9-C458CE615650,1999,1/4/1999 0:00,NaN,REGISTRATION,UNDETERMINED,47531,ROCKY MOUNTAIN GOVERNMENT RELATIONS,NaN,"14980 CAMPROBBER CT\nCOLORADO SPRINGS, CO 80908",USA,USA,SPACE PLAZA GROUP,NaN,24,False,JEFFREY G. CRANK,False,USA,USA,COLORADO,COLORADO,DEFENSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,A9623E72-F519-4927-AFD9-C458CE615650,1999,1/4/1999 0:00,NaN,REGISTRATION,UNDETERMINED,47531,ROCKY MOUNTAIN GOVERNMENT RELATIONS,NaN,"14980 CAMPROBBER CT\nCOLORADO SPRINGS, CO 80908",USA,USA,SPACE PLAZA GROUP,NaN,24,False,JEFFREY G. CRANK,False,USA,USA,COLORADO,COLORADO,REAL ESTATE/LAND USE/CONSERVATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,A9623E72-F519-4927-AFD9-C458CE615650,1999,1/4/1999 0:00,NaN,REGISTRATION,UNDETERMINED,47531,ROCKY MOUNTAIN GOVERNMENT RELATIONS,NaN,"14980 CAMPROBBER CT\nCOLORADO SPRINGS, CO 80908",USA,USA,SPACE PLAZA GROUP,NaN,24,False,JEFFREY G. CRANK,False,USA,USA,COLORADO,COLORADO,NaN,NaN,"CRANK, JEFFREY G",COVERED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,30EFF054-C6C9-4236-BAEE-1452F29CB207,1999,1/5/1999 0:00,NaN,REGISTRATION,UNDETERMINED,37152,"STURBITTS, CHARLENE A.",NaN,"3211 N. WOODROW STREET\nARLINGTON, VA 22207",USA,USA,FREE TRADE LUMBER COUNCIL,NaN,48,False,CHARLENE A. STURBITTS,False,CANADA,CANADA,NaN,NaN,NATURAL RESOURCES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Exploration

In [6]:
lob.nunique()

Unnamed: 0                              650916
ID                                     1263473
Year                                        21
Received                                828917
Amount                                   25876
Type                                        54
Period                                       7
RegistrantID                             11822
RegistrantName                           12675
GeneralDescription                        5109
Address                                  13388
RegistrantCountry                           24
RegistrantPPBCountry                        27
ClientName                               70327
GeneralDescription2                      34017
ClientID                                  6008
SelfFiler                                    2
ContactFullname                          19657
IsStateOrLocalGov                            2
ClientCountry                              151
ClientPPBCountry                           150
ClientState  

In [17]:
# ID is a unique code that matches registrant to client
lob[lob['ID'] == '30EFF054-C6C9-4236-BAEE-1452F29CB207'].head()

,Unnamed: 0,ID,Year,Received,Amount,Type,Period,RegistrantID,RegistrantName,GeneralDescription,Address,RegistrantCountry,RegistrantPPBCountry,ClientName,GeneralDescription2,ClientID,SelfFiler,ContactFullname,IsStateOrLocalGov,ClientCountry,ClientPPBCountry,ClientState,ClientPPBState,Code,SpecificIssue,LobbyistName,LobbyistCoveredGovPositionIndicator,OfficialPosition,GovEntityName,ForeignEntityName,ForeignEntityCountry,ForeignEntityPPBcountry,ForeignEntityContribution,ForeignEntityOwnershipPercentage,ForeignEntityStatus,AffiliatedOrgName,AffiliatedOrgCountry,AffiliatedOrgPPBCcountry,ActivityInformation,AffiliatedOrgsURL,RegistrationEffectiveDate,TerminationEffectiveDate
4,4,30EFF054-C6C9-4236-BAEE-1452F29CB207,1999,1/5/1999 0:00,NaN,REGISTRATION,UNDETERMINED,37152,"STURBITTS, CHARLENE A.",NaN,"3211 N. WOODROW STREET\nARLINGTON, VA 22207",USA,USA,FREE TRADE LUMBER COUNCIL,NaN,48,False,CHARLENE A. STURBITTS,False,CANADA,CANADA,NaN,NaN,NATURAL RESOURCES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,30EFF054-C6C9-4236-BAEE-1452F29CB207,1999,1/5/1999 0:00,NaN,REGISTRATION,UNDETERMINED,37152,"STURBITTS, CHARLENE A.",NaN,"3211 N. WOODROW STREET\nARLINGTON, VA 22207",USA,USA,FREE TRADE LUMBER COUNCIL,NaN,48,False,CHARLENE A. STURBITTS,False,CANADA,CANADA,NaN,NaN,TRADE (DOMESTIC/FOREIGN),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,30EFF054-C6C9-4236-BAEE-1452F29CB207,1999,1/5/1999 0:00,NaN,REGISTRATION,UNDETERMINED,37152,"STURBITTS, CHARLENE A.",NaN,"3211 N. WOODROW STREET\nARLINGTON, VA 22207",USA,USA,FREE TRADE LUMBER COUNCIL,NaN,48,False,CHARLENE A. STURBITTS,False,CANADA,CANADA,NaN,NaN,LABOR ISSUES/ANTITRUST/WORKPLACE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,30EFF054-C6C9-4236-BAEE-1452F29CB207,1999,1/5/1999 0:00,NaN,REGISTRATION,UNDETERMINED,37152,"STURBITTS, CHARLENE A.",NaN,"3211 N. WOODROW STREET\nARLINGTON, VA 22207",USA,USA,FREE TRADE LUMBER COUNCIL,NaN,48,False,CHARLENE A. STURBITTS,False,CANADA,CANADA,NaN,NaN,AGRICULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,30EFF054-C6C9-4236-BAEE-1452F29CB207,1999,1/5/1999 0:00,NaN,REGISTRATION,UNDETERMINED,37152,"STURBITTS, CHARLENE A.",NaN,"3211 N. WOODROW STREET\nARLINGTON, VA 22207",USA,USA,FREE TRADE LUMBER COUNCIL,NaN,48,False,CHARLENE A. STURBITTS,False,CANADA,CANADA,NaN,NaN,HOUSING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Seeing how report types change over time
Year = 2017 #@param ["1999", "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"] {type:"raw"}
variable_name = ""
lob[lob['Year'] == Year]['Type'].value_counts(normalize=True)

FOURTH QUARTER REPORT                                 0.222777
THIRD QUARTER REPORT                                  0.220662
SECOND QUARTER REPORT                                 0.217021
FIRST QUARTER REPORT                                  0.207387
REGISTRATION                                          0.032738
FIRST QUARTER AMENDMENT                               0.015454
SECOND QUARTER AMENDMENT                              0.013194
THIRD QUARTER AMENDMENT                               0.012180
FOURTH QUARTER AMENDMENT                              0.012055
FOURTH QUARTER TERMINATION                            0.009216
REGISTRATION AMENDMENT                                0.004935
SECOND QUARTER TERMINATION                            0.004476
THIRD QUARTER TERMINATION                             0.004332
FIRST QUARTER TERMINATION                             0.004015
FIRST QUARTER (NO ACTIVITY)                           0.003956
THIRD QUARTER (NO ACTIVITY)                           0

In [23]:
# See type count over all periods
lob['Type'].value_counts(normalize=True)

SECOND QUARTER REPORT                                 0.153743
THIRD QUARTER REPORT                                  0.151425
FIRST QUARTER REPORT                                  0.150839
FOURTH QUARTER REPORT                                 0.145594
MID-YEAR REPORT                                       0.122455
YEAR-END REPORT                                       0.115649
REGISTRATION                                          0.044635
FIRST QUARTER AMENDMENT                               0.010758
SECOND QUARTER AMENDMENT                              0.009539
YEAR-END TERMINATION                                  0.008916
REGISTRATION AMENDMENT                                0.008573
FOURTH QUARTER AMENDMENT                              0.008110
THIRD QUARTER AMENDMENT                               0.008012
MID-YEAR TERMINATION                                  0.006553
FOURTH QUARTER TERMINATION                            0.006286
MID-YEAR AMENDMENT                                    0

In [26]:
# all 2nd Q reports have the same date for Received column
lob[lob['Type'] == 'SECOND QUARTER REPORT'].head()

,Unnamed: 0,ID,Year,Received,Amount,Type,Period,RegistrantID,RegistrantName,GeneralDescription,Address,RegistrantCountry,RegistrantPPBCountry,ClientName,GeneralDescription2,ClientID,SelfFiler,ContactFullname,IsStateOrLocalGov,ClientCountry,ClientPPBCountry,ClientState,ClientPPBState,Code,SpecificIssue,LobbyistName,LobbyistCoveredGovPositionIndicator,OfficialPosition,GovEntityName,ForeignEntityName,ForeignEntityCountry,ForeignEntityPPBcountry,ForeignEntityContribution,ForeignEntityOwnershipPercentage,ForeignEntityStatus,AffiliatedOrgName,AffiliatedOrgCountry,AffiliatedOrgPPBCcountry,ActivityInformation,AffiliatedOrgsURL,RegistrationEffectiveDate,TerminationEffectiveDate
2545685,163902,881C7053-429D-410F-A30D-01CCE07941B5,2008,3/23/2008 11:31,239500.0,SECOND QUARTER REPORT,2nd Quarter (Apr 1 - June 30),9897,College of American Pathologists,NaN,"1350 I STREET, NW, #590\nWASHINGTON, DC 22039",USA,USA,COLLEGE OF AMERICAN PATHOLOGISTS,NaN,12,True,Michael Giuliani,True,USA,USA,DISTRICT OF COLUMBIA,DISTRICT OF COLUMBIA,NaN,NaN,"SCOTT, JOHN",COVERED,"Vice President, Division of Advocacy",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2545686,163903,881C7053-429D-410F-A30D-01CCE07941B5,2008,3/23/2008 11:31,239500.0,SECOND QUARTER REPORT,2nd Quarter (Apr 1 - June 30),9897,College of American Pathologists,NaN,"1350 I STREET, NW, #590\nWASHINGTON, DC 22039",USA,USA,COLLEGE OF AMERICAN PATHOLOGISTS,NaN,12,True,Michael Giuliani,True,USA,USA,DISTRICT OF COLUMBIA,DISTRICT OF COLUMBIA,NaN,NaN,"BELL, DENISE J",COVERED,"Director, Federal Legislative Affairs",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2545687,163904,881C7053-429D-410F-A30D-01CCE07941B5,2008,3/23/2008 11:31,239500.0,SECOND QUARTER REPORT,2nd Quarter (Apr 1 - June 30),9897,College of American Pathologists,NaN,"1350 I STREET, NW, #590\nWASHINGTON, DC 22039",USA,USA,COLLEGE OF AMERICAN PATHOLOGISTS,NaN,12,True,Michael Giuliani,True,USA,USA,DISTRICT OF COLUMBIA,DISTRICT OF COLUMBIA,NaN,NaN,"SPANGLER, REBECCA",COVERED,"Assistant Director, Federal Legislative Affairs",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2545688,163905,881C7053-429D-410F-A30D-01CCE07941B5,2008,3/23/2008 11:31,239500.0,SECOND QUARTER REPORT,2nd Quarter (Apr 1 - June 30),9897,College of American Pathologists,NaN,"1350 I STREET, NW, #590\nWASHINGTON, DC 22039",USA,USA,COLLEGE OF AMERICAN PATHOLOGISTS,NaN,12,True,Michael Giuliani,True,USA,USA,DISTRICT OF COLUMBIA,DISTRICT OF COLUMBIA,NaN,NaN,"DONNELLAN, CHRISTOPHER",COVERED,"Assistant Director, Federal Legislative Affairs",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2545689,163906,881C7053-429D-410F-A30D-01CCE07941B5,2008,3/23/2008 11:31,239500.0,SECOND QUARTER REPORT,2nd Quarter (Apr 1 - June 30),9897,College of American Pathologists,NaN,"1350 I STREET, NW, #590\nWASHINGTON, DC 22039",USA,USA,COLLEGE OF AMERICAN PATHOLOGISTS,NaN,12,True,Michael Giuliani,True,USA,USA,DISTRICT OF COLUMBIA,DISTRICT OF COLUMBIA,NaN,NaN,"SHERIN, CHRISTOPHER",COVERED,"Legislative Assistant, Federal Legislative Aff...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2545690,163907,881C7053-429D-410F-A30D-01CCE07941B5,2008,3/23/2008 11:31,239500.0,SECOND QUARTER REPORT,2nd Quarter (Apr 1 - June 30),9897,College of American Pathologists,NaN,"1350 I STREET, NW, #590\nWASHINGTON, DC 22039",USA,USA,COLLEGE OF AMERICAN PATHOLOGISTS,NaN,12,True,Michael Giuliani,True,USA,USA,DISTRICT OF COLUMBIA,DISTRICT OF COLUMBIA,MEDICAL/DISEASE RESEARCH/CLINICAL LABS,Cytology Proficiency Testing- Support legislat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2545691,163908,881C7053-429D-410F-A30D-01CCE07941B5,2008,3/23/2008 11:31,239500.0,SECOND QUARTER REPORT,2nd Quarter (Apr 1 - June 30),9897,College of American Pathologists,NaN,"1350 I STREET, NW, #590\nWASHINGTON, DC 22039",USA,USA,COLLEGE OF AMERICAN PATHOLOGISTS,NaN,12,True,Michael Giuliani,True,USA,USA,DISTRICT OF COLUMBIA,DISTRICT OF COLUMBIA,MEDICARE/MEDICAID,Sustainable Gro

In [28]:
# period is very similar to type
lob['Period'].value_counts(normalize=True)

2nd Quarter (Apr 1 - June 30)    0.171107
1st Quarter (Jan 1 - Mar 31)     0.169769
3rd Quarter (July 1 - Sep 30)    0.166744
4th Quarter (Oct 1 - Dec 31)     0.164419
Mid-Year (Jan 1 - Jun 30)        0.139892
Year-End (July 1 - Dec 31)       0.134787
UNDETERMINED                     0.053281
Name: Period, dtype: float64

In [31]:
# need to clean up general description --  many of these should be combined
pd.DataFrame(lob['GeneralDescription'].value_counts())

,GeneralDescription
Law Firm,548498
Law firm,503380
law firm,278902
Government Relations,192791
Government Relations Consultant,142048
Government Relations Firm,111432
Lobbying firm,99426
Lobbying,97499
lobbying firm,94915
Government Affairs,91908


In [0]:
# can ommit official position
pd.DataFrame(lob['OfficialPosition'].value_counts()).head(35)

In [19]:
# Clean up goventityname for productization
pd.DataFrame(lob['GovEntityName'].value_counts()).head(30)

,GovEntityName
HOUSE OF REPRESENTATIVES,909766
SENATE,901211
"Defense, Dept of (DOD)",64160
"Health & Human Services, Dept of (HHS)",63977
Environmental Protection Agency (EPA),50831
"Transportation, Dept of (DOT)",49459
"Treasury, Dept of",49013
"Agriculture, Dept of (USDA)",47067
White House Office,44738
"Commerce, Dept of (DOC)",44434


In [22]:
# need to find out what this is
pd.DataFrame(lob['ActivityInformation'].value_counts())

,ActivityInformation
A,1498148
B,53614


In [20]:
lob.head(1)

,Unnamed: 0,ID,Year,Received,Amount,Type,Period,RegistrantID,RegistrantName,GeneralDescription,Address,RegistrantCountry,RegistrantPPBCountry,ClientName,GeneralDescription2,ClientID,SelfFiler,ContactFullname,IsStateOrLocalGov,ClientCountry,ClientPPBCountry,ClientState,ClientPPBState,Code,SpecificIssue,LobbyistName,LobbyistCoveredGovPositionIndicator,OfficialPosition,GovEntityName,ForeignEntityName,ForeignEntityCountry,ForeignEntityPPBcountry,ForeignEntityContribution,ForeignEntityOwnershipPercentage,ForeignEntityStatus,AffiliatedOrgName,AffiliatedOrgCountry,AffiliatedOrgPPBCcountry,ActivityInformation,AffiliatedOrgsURL,RegistrationEffectiveDate,TerminationEffectiveDate
0,0,83792D41-7B9E-40E4-87AA-B7828C79DCAC,1999,1/1/1999 0:00,NaN,REGISTRATION,UNDETERMINED,24624,MCLEOD WATKINSON & MILLER,Lobbying and government affairs,ONE MASSACHUSETTS AVE. NW\nSUITE 800\nWASHINGT...,USA,USA,CALIFORNIA AVOCADO COMMISSION,NaN,36,False,MICHAEL R. MCLEOD,False,USA,USA,CALIFORNIA,CALIFORNIA,AGRICULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Notes:
# - change all the quarterly reports to "Quarter", mid-year reports to "Semi-Annual" and year-end reports to "Annual"
# - drop outliers in amount
# - standardize text of registrant name, client name, goventityname and general desciption (trim data down as much as possible first)
# - get rid of Amounts that are NaN and 0
# - 

# What is SelfFiler, LobbyistCoveredGovPositionIndicator and ActivityInformation?

In [4]:
# wrangle and feature engineer function

def wrangle(X):
    X = X.copy()
    
    # drop extra columns
    X = X.drop(columns=['Unnamed: 0', 'ID', 'Received', 'Period', 'RegistrantID', 'Address', 'RegistrantPPBCountry', 
                        'GeneralDescription2', 'ClientID', 'ContactFullname', 'ClientPPBCountry', 'ClientPPBState', 'SpecificIssue',
                        'LobbyistName', 'OfficialPosition', 'ForeignEntityName', 'ForeignEntityCountry', 'ForeignEntityPPBcountry', 
                        'ForeignEntityContribution', 'ForeignEntityOwnershipPercentage', 	'ForeignEntityStatus', 
                        'AffiliatedOrgName', 'AffiliatedOrgCountry', 'AffiliatedOrgPPBCcountry', 'AffiliatedOrgsURL', 
                        'RegistrationEffectiveDate', 'TerminationEffectiveDate'])
    
    # subset where Amount is not NaN or 0
    X = X.dropna(subset=['Amount'])
    X = X[X['Amount'] != 0]
    
    # subset of specific report types
    X = X[(X['Type'] == 'SECOND QUARTER REPORT') | (X['Type'] == 'THIRD QUARTER REPORT') | 
          (X['Type'] == 'FIRST QUARTER REPORT') | (X['Type'] == 'FOURTH QUARTER REPORT')]
    
    # get rid of outliers -- subset of amounts that fall within 3 standard deviations of the mean
    X = X[np.abs(X['Amount']-X['Amount'].mean()) <= (3*X['Amount'].std())]
    
    # format report type
    X['Type'] = X['Type'].map({
        'FIRST QUARTER REPORT': '1st Quarter',
        'SECOND QUARTER REPORT': '2nd Quarter',
        'THIRD QUARTER REPORT': '3rd Quarter',
        'FOURTH QUARTER REPORT': '4th Quarter'})
    
    # function to clean registrant name
    def clean_registrant(x):
        rm_inbetween_parens = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", x)
        clean_string = rm_inbetween_parens.replace('(', '').replace(')', '').replace('.', '').replace(',', '').strip().title()
        return clean_string
    
    # apply above function
    X['RegistrantName'] = X['RegistrantName'].apply(clean_registrant)
    
    # tokenize strings
    X['RegistrantName'] = X['RegistrantName'].str.split()

    # make dictionary of stop words (i.e. words to remove)
    reg_stop_words = {'Llp', 'Pllc', 'Inc', 'Llc'}

    # function that removes stop words
    def rm_stop_words(s):
        s = [w for w in s if not w in reg_stop_words]
        return s

    # apply function to client names
    X['RegistrantName'] = X['RegistrantName'].apply(rm_stop_words)

    # join list of strings into a single string
    X['RegistrantName'] = X['RegistrantName'].str.join(" ")
    
    # format general description
    X['GeneralDescription'] = X['GeneralDescription'].str.title()
    X['GeneralDescription'] = X['GeneralDescription'].str.replace('.', '')
    X['GeneralDescription'] = X['GeneralDescription'].str.replace('Firm', '')
    X['GeneralDescription'] = X['GeneralDescription'].str.strip()
    X['GeneralDescription'] = X['GeneralDescription'].str.replace('Consultant', 'Consulting')
    X['GeneralDescription'] = X['GeneralDescription'].str.replace('And', '&')
    X['GeneralDescription'] = X['GeneralDescription'].str.replace('Government Relations & Consulting', 'Government Relations Consulting')
    X['GeneralDescription'] = X['GeneralDescription'].str.replace('Government Relations Consulting', 'Government Relations')
    X['GeneralDescription'] = X['GeneralDescription'].str.replace('Government Affairs Consulting', 'Government Affairs')
    X['GeneralDescription'] = X['GeneralDescription'].str.replace('Government Affairs Services', 'Government Affairs')
    top10 = X['GeneralDescription'].value_counts().head(10).index.tolist()
    
    # function to return top10 general description or 'Other'
    def top10_desc(s):
        if s in top10:
            return s
        else:
            return 'Other'
    # apply above function to general description
    X['GeneralDescription'] = X['GeneralDescription'].apply(top10_desc)
    
    # function to determine if registrant country is US or other
    def is_us(s):    
        if s == 'USA':
            return 'USA'
        else:
            return 'Other'
    # apply above function to registrant country
    X['RegistrantCountry'] = X['RegistrantCountry'].apply(is_us)
    
    # function to client name text
    X['ClientName'] = X['ClientName'].str.title()
    def clean_client(x):
        rm_inbetween_parens = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", x)
        clean_string = rm_inbetween_parens.replace('(', '').replace(')', '').replace('.', '').replace(',', '').strip().title()
        return clean_string
    # apply above function to client name
    X['ClientName'] = X['ClientName'].apply(clean_client)
    
    # tokenize strings
    X['ClientName'] = X['ClientName'].str.split()

    # make dictionary of stop words (i.e. words to remove)
    cli_stop_words = {'Corp', 'Corporation', 'Inc', 'Incorporated', 'Co', 'Company', 'Plc', 'Group', 'Llc', 'Llp', 'Lp', 'The', 'Usa'}

    # function that removes stop words
    def rm_stop_words(s):
        s = [w for w in s if not w in cli_stop_words]
        return s

    # apply function to client names
    X['ClientName'] = X['ClientName'].apply(rm_stop_words)

    # join list of strings into a single string
    X['ClientName'] = X['ClientName'].str.join(" ")
    
    # format client names
    X['ClientName'] = X['ClientName'].str.replace('Wal-Mart Stores', 'Wal-Mart')
    X['ClientName'] = X['ClientName'].str.replace('Verizon Communications And Various Subsidiaries', 'Verizon')
    X['ClientName'] = X['ClientName'].str.replace('Eli Lilly And', 'Eli Lilly')
    X['ClientName'] = X['ClientName'].str.replace('At&T Services', 'At&T')
    X['ClientName'] = X['ClientName'].str.replace('Johnson & Johnson Services', 'Johnson & Johnson')
    X['ClientName'] = X['ClientName'].str.replace('Citigroup Management', 'Citigroup')
    X['ClientName'] = X['ClientName'].str.replace('Bp America', 'Bp')
    X['ClientName'] = X['ClientName'].str.replace('Verizon Communications', 'Verizon')
    X['ClientName'] = X['ClientName'].str.replace('Merck &', 'Merck & Co')
    X['ClientName'] = X['ClientName'].str.replace('Time Warner Cable', 'Time Warner')
    X['ClientName'] = X['ClientName'].str.replace('Toyota Motor North America', 'Toyota')
    X['ClientName'] = X['ClientName'].str.replace('Amazon Corporate', 'Amazon')
    X['ClientName'] = X['ClientName'].str.replace('Chevron U S A', 'Chevron')
    X['ClientName'] = X['ClientName'].str.replace('Jpmorgan Chase &', 'Jpmorgan Chase')
    X['ClientName'] = X['ClientName'].str.replace('Mcdonald\'S', 'Mcdonald\'s')
    X['ClientName'] = X['ClientName'].str.replace('Motorola Solutions', 'Motorola')
    X['ClientName'] = X['ClientName'].str.replace('Amazoncom', 'Amazon')
    X['ClientName'] = X['ClientName'].str.replace('Hp', 'Hewlett-Packard')
    X['ClientName'] = X['ClientName'].str.replace('At&T And Its Affiliates', 'At&T')
    X['ClientName'] = X['ClientName'].str.replace('Nissan North America', 'Nissan')
    X['ClientName'] = X['ClientName'].str.replace('Berkshire Hathaway Energy', 'Berkshire Hathaway')
    X['ClientName'] = X['ClientName'].str.replace('Honda North America', 'Honda')
    X['ClientName'] = X['ClientName'].str.replace('Oracle America', 'Oracle')
    X['ClientName'] = X['ClientName'].str.replace('Charles Schwab &', 'Charles Schwab')
    X['ClientName'] = X['ClientName'].str.replace('Panasonic Of North America', 'Panasonic')
    X['ClientName'] = X['ClientName'].str.replace('Deere &', 'Deere & Company')
    X['ClientName'] = X['ClientName'].str.replace('Wells Fargo &', 'Wells Fargo')
    X['ClientName'] = X['ClientName'].str.replace('Verizon Wireless', 'Verizon')
    X['ClientName'] = X['ClientName'].str.replace('Exxonmobil', 'Exxon Mobil')
    X['ClientName'] = X['ClientName'].str.replace('Prudential Insurance Of America', 'Prudential')
    X['ClientName'] = X['ClientName'].str.replace('Salesforcecom', 'Salesforce')
    X['ClientName'] = X['ClientName'].str.replace('Sprint Nextel', 'Sprint')
    X['ClientName'] = X['ClientName'].str.replace('Starbucks Coffee', 'Starbucks')
    X['ClientName'] = X['ClientName'].str.replace('Jpmorgan Chase Holdings', 'Jpmorgan Chase')
    X['ClientName'] = X['ClientName'].str.replace('Dell', 'Dell Technologies')
    X['ClientName'] = X['ClientName'].str.replace('Eli Lilly &', 'Eli Lilly')
    X['ClientName'] = X['ClientName'].str.replace('Federal Express', 'Fedex')
    
    # list of client countries
    other_countries = X['ClientCountry'].value_counts().index.tolist()[1:]
    
    # function to determine USA or "Other"
    def us_or_other(s):
        if s in other_countries:
            return 'Other'
        else:
            return 'USA'
    # apply above function to client country
    X['ClientCountry'] = X['ClientCountry'].apply(us_or_other)
    
    # format client state
    X['ClientState'] = X['ClientState'].str.title()
    X['ClientState'] = X['ClientState'].fillna('Outside US')
    
    # format code/issue
    X['Code'] = X['Code'].str.title()
    X['Code'] = X['Code'].fillna('Undetermined')
    
    # format lobbyist covered
    X['LobbyistCoveredGovPositionIndicator'] = X['LobbyistCoveredGovPositionIndicator'].str.title()
    X['LobbyistCoveredGovPositionIndicator'] = X['LobbyistCoveredGovPositionIndicator'].fillna('Undetermined')
    
    # gov entity name
    X['GovEntityName'] = X['GovEntityName'].str.title()
    X['GovEntityName'] = X['GovEntityName'].str.strip()
    X['GovEntityName'] = X['GovEntityName'].fillna('Undetermined')
    
    # function to clean gov entity text 
    def clean_gov_entity(x):
        rm_inbetween_parens = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", x)
        clean_string = clean_string = rm_inbetween_parens.replace('(', '').replace(')', '')
        return clean_string
    # apply above function
    X['GovEntityName'] = X['GovEntityName'].apply(clean_gov_entity)
    
    # reformat gov entity names
    X['GovEntityName'] = X['GovEntityName'].str.replace('Health & Human Services, Dept Of', 'Dept of Health & Human Services')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Defense, Dept Of', 'Dept of Defense')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Environmental Protection Agency', 'Environmental Protection Agency')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Transportation, Dept Of', 'Dept of Transportation')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Treasury, Dept Of', 'Dept of Treasury')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Agriculture, Dept Of', 'Dept of Agriculture')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Centers For Medicare And Medicaid Services', 'Centers for Medicare and Medicaid Services')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Energy, Dept Of', 'Dept of Energy')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Commerce, Dept Of', 'Dept of Commerce')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Office Of Management & Budget, Dept Of', 'Office of Management & Budget')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Interior, Dept Of', 'Dept of Interior')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Homeland Security, Dept Of', 'Dept of Homeland Security')
    X['GovEntityName'] = X['GovEntityName'].str.replace('State, Dept Of', 'Dept of State')
    X['GovEntityName'] = X['GovEntityName'].str.replace('U.S. Trade Representative', 'US Trade Representative')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Executive Office Of The President', 'White House')
    X['GovEntityName'] = X['GovEntityName'].str.replace('White House Office', 'White House')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Army, Dept Of', 'Dept of Army')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Labor, Dept Of', 'Dept of Labor')
    X['GovEntityName'] = X['GovEntityName'].str.replace('Education, Dept Of', 'Dept of Education')
    X['GovEntityName'] = X['GovEntityName'].str.strip()
    
    # list of top 20 gov entity names
    top20_entity = X['GovEntityName'].value_counts().head(21).index.tolist()[1:]
    
    # function that returns gov entity name if in top 20, else returns 'Other'
    def top20_ent(s):
        if s in top20_entity:
            return s
        else:
            return 'Other'
    # apply above function
    X['GovEntityName'] = X['GovEntityName'].apply(top20_ent)
    
    # format activity information
    X['ActivityInformation'] = X['ActivityInformation'].fillna('Undetermined')
    
    return X

In [5]:
# apply wrangle function
new_lob = wrangle(lob)

In [6]:
new_lob.head()

,Year,Amount,Type,RegistrantName,GeneralDescription,RegistrantCountry,ClientName,SelfFiler,IsStateOrLocalGov,ClientCountry,ClientState,Code,LobbyistCoveredGovPositionIndicator,GovEntityName,ActivityInformation
2540762,2008,20000.0,1st Quarter,Darlene Richeson & Associates,Lobbying,USA,Verizon,False,False,USA,Undetermined,Undetermined,Not Covered,Other,Undetermined
2540763,2008,20000.0,1st Quarter,Darlene Richeson & Associates,Lobbying,USA,Verizon,False,False,USA,Undetermined,Telecommunications,Undetermined,Other,Undetermined
2540764,2008,20000.0,1st Quarter,Darlene Richeson & Associates,Lobbying,USA,Verizon,False,False,USA,Undetermined,Undetermined,Undetermined,House Of Representatives,Undetermined
2540765,2008,30000.0,1st Quarter,Darlene Richeson & Associates,Lobbying,USA,Johnson & Johnson,False,False,USA,District Of Columbia,Undetermined,Not Covered,Other,Undetermined
2540766,2008,30000.0,1st Quarter,Darlene Richeson & Associates,Lobbying,USA,Johnson & Johnson,False,False,USA,District Of Columbia,Health Issues,Undetermined,Other,Undetermined


In [7]:
new_lob.isnull().sum()

Year                                   0
Amount                                 0
Type                                   0
RegistrantName                         0
GeneralDescription                     0
RegistrantCountry                      0
ClientName                             0
SelfFiler                              0
IsStateOrLocalGov                      0
ClientCountry                          0
ClientState                            0
Code                                   0
LobbyistCoveredGovPositionIndicator    0
GovEntityName                          0
ActivityInformation                    0
dtype: int64

In [17]:
new_lob[new_lob['ClientState'] == 'Outside US'].head()

,Year,Amount,Type,RegistrantName,GeneralDescription,RegistrantCountry,ClientName,SelfFiler,IsStateOrLocalGov,ClientCountry,ClientState,Code,LobbyistCoveredGovPositionIndicator,GovEntityName,ActivityInformation
2540768,2008,30000.0,1st Quarter,Sonoran Policy Group,Lobbying,USA,Applied Identity,False,False,Other,Outside US,Undetermined,Not Covered,Other,Undetermined
2540769,2008,30000.0,1st Quarter,Sonoran Policy Group,Lobbying,USA,Applied Identity,False,False,Other,Outside US,Undetermined,Not Covered,Other,Undetermined
2540770,2008,30000.0,1st Quarter,Sonoran Policy Group,Lobbying,USA,Applied Identity,False,False,Other,Outside US,Homeland Security,Undetermined,Other,Undetermined
2540771,2008,30000.0,1st Quarter,Sonoran Policy Group,Lobbying,USA,Applied Identity,False,False,Other,Outside US,Undetermined,Undetermined,Other,Undetermined
2540772,2008,30000.0,1st Quarter,Sonoran Policy Group,Lobbying,USA,Applied Identity,False,False,Other,Outside US,Undetermined,Undetermined,Other,Undetermined


In [18]:
new_lob['ClientState'].value_counts()

District Of Columbia        1483167
Virginia                     366919
California                   332042
Outside US                   282155
New York                     188035
Texas                        169841
Florida                      123338
Illinois                     117558
Maryland                     105391
Pennsylvania                 102105
Massachusetts                 93082
Ohio                          82746
New Jersey                    76376
Washington                    75310
Georgia                       59344
Colorado                      56348
North Carolina                51994
Michigan                      45627
Missouri                      45199
Connecticut                   43230
Minnesota                     41665
Indiana                       37553
Arizona                       36879
Louisiana                     36657
Wisconsin                     35659
Undetermined                  34858
Oregon                        34540
Tennessee                   

In [19]:
new_lob.shape

(4517063, 15)

In [21]:
X = new_lob.drop(columns='Amount')
y = new_lob['Amount'].values

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

In [ ]:
# save results of new lobbying data
# new_lob.to_csv('clean_lobby.csv')

In [23]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3026432, 14), (1490631, 14), (3026432,), (1490631,))

In [56]:
# trimming unimportant columns
X_trimmed = new_lob.drop(columns=['Amount', 'ActivityInformation', 'LobbyistCoveredGovPositionIndicator', 
                                  'Code', 'GovEntityName', 'RegistrantCountry'])

X_train_trimmed, X_test_trimmed, y_train, y_test = train_test_split(X_trimmed, y, test_size=.33, random_state=42)

X_train_trimmed.shape, X_test_trimmed.shape, y_train.shape, y_test.shape

((3026432, 9), (1490631, 9), (3026432,), (1490631,))

In [60]:
X_train_trimmed.head()

,Year,Type,RegistrantName,GeneralDescription,ClientName,SelfFiler,IsStateOrLocalGov,ClientCountry,ClientState
7644757,2017,2nd Quarter,The Keelen Group,Other,Las Vegas Sands,False,True,USA,Nevada
2797553,2008,2nd Quarter,National Council Of La Raza,Other,National Council Of La Raza,True,True,USA,District Of Columbia
6007570,2014,1st Quarter,Palmetto Group,Other,Koch Companies Public Sector,False,True,USA,District Of Columbia
5767106,2013,3rd Quarter,The Rogich Communications Group,Other,Cigar Rights Of America,False,True,USA,District Of Columbia
5397475,2012,4th Quarter,Newton Dillard,Government Relations,National Urban League,False,True,USA,District Of Columbia


## Random Forest Regressor Model

In [57]:
# make pipeline
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    StandardScaler(),
    RandomForestRegressor(n_estimators=250, n_jobs=16))

# fit pipeline
pipeline.fit(X_train_trimmed, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype bool, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


Pipeline(memory=None,
     steps=[('ordinalencoder', OrdinalEncoder(cols=['Type', 'RegistrantName', 'GeneralDescription', 'ClientName', 'ClientCountry', 'ClientState'],
        drop_invariant=False, handle_missing='value',
        handle_unknown='value',
        mapping=[{'col': 'Type', 'mapping': 2nd Quarter    1
1st Quarter...mators=250, n_jobs=16,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [58]:
# Train dataset predictions
y_pred_train = pipeline.predict(X_train_trimmed)
mae = mean_absolute_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)
print('Mean Absolute Error:', mae)
print('R-squared:', r2)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype bool, int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


Mean Absolute Error: 380.1438886364285
R-squared: 0.9992968936189953


In [59]:
# Test dataset predictions
y_pred_test = pipeline.predict(X_test_trimmed)
mae = mean_absolute_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)
print('Mean Absolute Error:', mae)
print('R-squared:', r2)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


Mean Absolute Error: 600.9674085426726
R-squared: 0.9991886743419096


In [38]:
# dataframe to make predictions
for_pred = X_test.copy()
for_pred['Amount'] = y_test
for_pred.head(50)

,Year,Type,RegistrantName,GeneralDescription,RegistrantCountry,ClientName,SelfFiler,IsStateOrLocalGov,ClientCountry,ClientState,Code,LobbyistCoveredGovPositionIndicator,GovEntityName,ActivityInformation,Amount
4079673,2010,2nd Quarter,Invariant,Government Relations,USA,Oz Management,False,True,Other,Outside US,Undetermined,Not Covered,Other,A,80000.0
3069476,2008,4th Quarter,Ogilvy Government Relations,Lobbying,USA,At&T,False,False,Other,Outside US,Telecommunications,Undetermined,Other,Undetermined,40000.0
8506068,2018,4th Quarter,Lockridge Grindal Nauen Pllp,Law,USA,City Of St Paul,False,True,USA,Oregon,Urban Development/Municipalities,Undetermined,Other,Undetermined,10000.0
3894404,2010,1st Quarter,The Home Depot,Other,USA,Home Depot,True,True,USA,District Of Columbia,Taxation/Internal Revenue Code,Undetermined,Other,Undetermined,245000.0
7175325,2016,2nd Quarter,Bavarian Nordic Washington Dc,Other,USA,Bavarian Nordic Washington Dc,True,True,USA,District Of Columbia,Undetermined,Undetermined,Dept of Health & Human Services,Undetermined,226000.0
5915108,2013,4th Quarter,Holland & Knight,Law,USA,City Of Tampa,False,True,USA,Florida,Environment/Superfund,Undetermined,Other,Undetermined,30000.0
6217080,2014,2nd Quarter,Thompson Consulting Group,Other,USA,Infoblox,False,True,USA,Washington,Homeland Security,Undetermined,Other,Undetermined,40000.0
8114578,2018,1st Quarter,Merck & Co,Other,USA,Merck & Co,True,True,USA,New Jersey,Undetermined,Not Covered,Other,A,3310000.0
5273323,2012,3rd Quarter,Community Bankers Association Of Illinois,Other,USA,Community Bankers Association Of Illinois,True,True,USA,Illinois,Undetermined,Undetermined,Other,Undetermined,50000.0
5062930,2012,1st Quarter,Entertainment Software Association,Other,USA,Entertainment Software Association,True,True,USA,District Of Columbia,Undetermined,Undetermined,Other,Undetermined,1082167.0


In [42]:
# single prediction dataframe
pred = pd.DataFrame(data=X_test.loc[7771285])
pred.T

,Year,Type,RegistrantName,GeneralDescription,RegistrantCountry,ClientName,SelfFiler,IsStateOrLocalGov,ClientCountry,ClientState,Code,LobbyistCoveredGovPositionIndicator,GovEntityName,ActivityInformation
7771285,2017,3rd Quarter,The Estell Group,Government Relations,USA,Charles Drew University Of Medicine And Science,False,True,USA,California,Undetermined,Undetermined,Senate,Undetermined


In [43]:
# seeing how prediction compares to actual amount
print(pipeline.predict(pred.T))
for_pred['Amount'].loc[pred.T.index]

[10000.]


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


7771285    10000.0
Name: Amount, dtype: float64

## Permutation Importances

In [54]:
# encode X_Train
encoder = ce.OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train)

# scale X_train
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)

# model instance
model = RandomForestRegressor(n_estimators=250, n_jobs=16)

# fit model, create permuter and fit permuter
model.fit(X_train_scaled, y_train)
permuter = PermutationImportance(model, scoring='neg_mean_absolute_error', n_iter=5, cv='prefit')
permuter.fit(X_train_scaled, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype bool, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


PermutationImportance(cv='prefit',
           estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=16,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
           n_iter=5, random_state=None, refit=True,
           scoring='neg_mean_absolute_error')

In [55]:
eli5.show_weights(permuter, top=None, feature_names=X_train.columns.tolist())

Weight,Feature
149866.5225 ± 227.6799,ClientName
132496.3121 ± 259.6134,SelfFiler
127853.2113 ± 285.9013,RegistrantName
83924.9967 ± 163.6643,ClientState
47302.0732 ± 134.7760,Year
24735.4231 ± 91.5527,Type
7696.3608 ± 11.7251,GeneralDescription
4607.7351 ± 38.7243,IsStateOrLocalGov
1300.2134 ± 9.2004,ClientCountry
609.5281 ± 6.4891,ActivityInformation


## Pickle Test

In [62]:
# save model
filename = 'lobby_contract_model.pickle'
pickle.dump(pipeline, open(filename, 'wb'))

In [63]:
# load model
loaded_model = pickle.load(open('lobby_contract_model.pickle', 'rb'))

In [ ]:
# test model

In [65]:
for_pred = X_test_trimmed.copy()
for_pred['Amount'] = y_test
for_pred.head(20)

,Year,Type,RegistrantName,GeneralDescription,ClientName,SelfFiler,IsStateOrLocalGov,ClientCountry,ClientState,Amount
4079673,2010,2nd Quarter,Invariant,Government Relations,Oz Management,False,True,Other,Outside US,80000.0
3069476,2008,4th Quarter,Ogilvy Government Relations,Lobbying,At&T,False,False,Other,Outside US,40000.0
8506068,2018,4th Quarter,Lockridge Grindal Nauen Pllp,Law,City Of St Paul,False,True,USA,Oregon,10000.0
3894404,2010,1st Quarter,The Home Depot,Other,Home Depot,True,True,USA,District Of Columbia,245000.0
7175325,2016,2nd Quarter,Bavarian Nordic Washington Dc,Other,Bavarian Nordic Washington Dc,True,True,USA,District Of Columbia,226000.0
5915108,2013,4th Quarter,Holland & Knight,Law,City Of Tampa,False,True,USA,Florida,30000.0
6217080,2014,2nd Quarter,Thompson Consulting Group,Other,Infoblox,False,True,USA,Washington,40000.0
8114578,2018,1st Quarter,Merck & Co,Other,Merck & Co,True,True,USA,New Jersey,3310000.0
5273323,2012,3rd Quarter,Community Bankers Association Of Illinois,Other,Community Bankers Association Of Illinois,True,True,USA,Illinois,50000.0
5062930,2012,1st Quarter,Entertainment Software Association,Other,Entertainment Software Association,True,True,USA,District Of Columbia,1082167.0


In [77]:
test_df = pd.DataFrame(data=for_pred.loc[7175325])
Test_df.T

,Year,Type,RegistrantName,GeneralDescription,ClientName,SelfFiler,IsStateOrLocalGov,ClientCountry,ClientState,Amount
7175325,2016,2nd Quarter,Bavarian Nordic Washington Dc,Other,Bavarian Nordic Washington Dc,True,True,USA,District Of Columbia,226000


In [78]:
print(loaded_model.predict(test_df.T.drop(columns='Amount')))
for_pred['Amount'].loc[test_df.T.index]

[226240.]


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


7175325    226000.0
Name: Amount, dtype: float64

In [ ]:
# pickle file works